In [ ]:
import torch
import random
import numpy as np
from pathlib import Path
import os
import sys

# Add parent directory to path so we can import project files in notebook
current_dir = os.path.dirname(os.path.realpath("__file__"))
lib_path = os.path.join(current_dir, "..")
sys.path.append(lib_path)

from runs.validate_model import validate_single_model
from models.nn_models import ExampleLSTM
from util.data import generate_settings, update_settings

# Autoreload
%load_ext autoreload
%autoreload 2

# Set random seed for reproducibility for random, numpy, and torch
seed = 16
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

model_name_base = "ExampleLSTM_test"
model_constructor = ExampleLSTM
base_args = {
    "init": {
        "n_ts_features": 9,
        "n_static_features": 1,
        "hidden_size": 32,
        "num_layers": 1,
    },

    "fit": {
        'batch_size': 16,
        'num_epochs': 150,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'optim_fn': torch.optim.Adam,
        'optim_kwargs': {"lr": 0.0001, 'weight_decay': 0.00001},
        'scheduler_fn': torch.optim.lr_scheduler.StepLR,
        'scheduler_kwargs': {"step_size": 1, "gamma": 1},
        'val_fraction': 0.1,
        'val_split_by_year': True,
        'seed': 16,
        'do_early_stopping': True,
    }
}

run_name_base = "example_hparam_run"

# Define hyperparameters to search over and their values
param_space = {
    "init.hidden_size": [16, 32, 64],
    "init.num_layers": [1, 2],
    "fit.optim_kwargs.lr": [0.00001, 0.0001],
    "fit.optim_kwargs.weight_decay": [0.00001],
}


settings = generate_settings(param_space, base_args)
print(f"Generated {len(settings)} settings to run.")

# Create base folder for run_name
Path(f"../output/runs/{run_name_base}").mkdir(parents=True, exist_ok=True)

# Shuffle settings
random.shuffle(settings)

# Run the hyperparameter search   
for i, setting in enumerate(settings):

    run_kwargs = update_settings(setting, base_args)

    print(f"Running {run_name_base} {i} with settings:")
    print(run_kwargs)

    result_df = validate_single_model(
        run_name_base + '_' + str(i),
        model_name_base + '_' + str(i),
        model_constructor,
        run_kwargs['init'],
        run_kwargs['fit'],
        dataset_name = "test_maize_us",
        test_years_to_leave_out=[2000],
    )

    # Save the results
    result_df.to_csv(f"../output/runs/{run_name_base}/{run_name_base}_{i}.csv", index=False)

In [ ]:
import pandas as pd
import re
from runs.run_benchmark import run_benchmark
from util.data import flatten_nested_dict, update_settings, unflatten_nested_dict

#run_name_base = "example_hparam_run"

# Loop over results and print the best one
best_result = None
best_result_idx = None
result_list = []
setting_list = []
for i, path in enumerate(Path(f"../output/runs/{run_name_base}").rglob("*.csv")):
    result_df = pd.read_csv(path)

    
    init_setting = result_df.iloc[0, 1]
    fit_setting = result_df.iloc[1, 1]

    # eval, but remove strings between < and > first

    init_setting = eval(re.sub(r'<.*?>', '', init_setting))
    fit_setting = eval(re.sub(r'<.*?>', "''", fit_setting))

    # Combine into one dict using flatten_nested_dict
    all_setting = flatten_nested_dict({"init": init_setting, "fit": fit_setting})

    if fit_setting['do_early_stopping']:
        val_loss = result_df.iloc[6, 1]
    else:
        val_loss = result_df.iloc[3, 1]
    print(f"Setting {i}: {val_loss}")
    result_list.append(val_loss)
    setting_list.append(all_setting)
    if best_result is None or val_loss < best_result:
        best_result = val_loss
        best_result_idx = i

print(f"Best result: {best_result} at index {best_result_idx}")
print(f"{pd.DataFrame(setting_list[best_result_idx], index=['Best settings:']).T}")


# Save best results to file
with open(f"../output/runs/{run_name_base}/best_settings_result.txt", "w") as f:
    f.write(f"Best result: {best_result} at index {best_result_idx}\n")

# Save best settings dict to file with pickle
import pickle
settings_to_save = setting_list[best_result_idx]
with open(f"../output/runs/{run_name_base}/best_settings_dict.pkl", "wb") as f:
    pickle.dump(settings_to_save, f)

# Unwrap settings from list of dicts to dict of lists
# flatten settings
new_settings = {}
for i, setting in enumerate(setting_list):
    
    flattened_setting = flatten_nested_dict(setting)
    for k, v in flattened_setting.items():
        # Take only final part of key, after last dot
        k = k.split('.')[-1]
        if k not in new_settings:
            new_settings[k] = []
        # Take only final part of key
        new_settings[k].append(v)

# Create dataframe from settings
df = pd.DataFrame(new_settings)


# Add val_loss column
df['val_loss'] = result_list
df['val_loss'] = df['val_loss'].astype(float)

# Filter out columns where all values are the same
df = df.loc[:, df.nunique() != 1]

# Make box plots of val loss for hidden_size	num_layers	lr
import seaborn as sns
import matplotlib.pyplot as plt


# For each hyperparameter that is not the same for all settings, plot a scatter plot of val_loss vs. that hyperparameter
# Plot them side by side
unique_columns = [col for col in df.columns if df[col].nunique() > 1 and df[col].name != 'val_loss']
num_cols = len(unique_columns)
print(unique_columns)
fig, axs = plt.subplots(1, num_cols, figsize=(5 * num_cols, 5))

col_idx = 0
for col in df.columns:
    if df[col].nunique() > 1 and df[col].name != 'val_loss':
        sns.scatterplot(x=col, y='val_loss', data=df, ax=axs[col_idx])
        col_idx += 1
plt.show()

# Same for box plots
num_cols = len([col for col in df.columns if df[col].nunique() > 1 and df[col].name != 'val_loss'])
fig, axs = plt.subplots(1, num_cols, figsize=(5 * num_cols, 5))

col_idx = 0
for col in df.columns:
    if df[col].nunique() > 1 and df[col].name != 'val_loss':
        sns.boxplot(x=col, y='val_loss', data=df, ax=axs[col_idx])
        col_idx += 1
plt.show()

In [ ]:
# Load best settings dict from file with pickle
import pickle
with open(f"../output/runs/{run_name_base}/best_settings_dict.pkl", "rb") as f:
    best_settings = pickle.load(f)


# Assert that the best settings are the same as the best settings found if settings exist
if best_result_idx is not None: 
    assert best_settings == settings[best_result_idx]


# Run the full benchmark for the best settings
run_kwargs = update_settings(best_settings, base_args)
result_df = run_benchmark(
    run_name_base + '_best',
    model_name_base + '_best',
    model_constructor,
    run_kwargs['init'],
    run_kwargs['fit'],
)
print(result_df)

In [ ]:
# Get the evaluation results
from runs.run_benchmark import _compute_evaluation_results
_compute_evaluation_results(run_name_base + '_best')